## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2051. The Category of Each Member in the Store (Medium)**

Table: Members

| Column Name | Type    |
|-------------|---------|
| member_id   | int     |
| name        | varchar |

member_id is the column with unique values for this table.
Each row of this table indicates the name and the ID of a member.
 
**Table: Visits**

| Column Name | Type |
|-------------|------|
| visit_id    | int  |
| member_id   | int  |
| visit_date  | date |

visit_id is the column with unique values for this table.
member_id is a foreign key (reference column) to member_id from the Members table.
Each row of this table contains information about the date of a visit to the store and the member who visited it.
 
**Table: Purchases**

| Column Name    | Type |
|----------------|------|
| visit_id       | int  |
| charged_amount | int  |

visit_id is the column with unique values for this table.
visit_id is a foreign key (reference column) to visit_id from the Visits table.
Each row of this table contains information about the amount charged in a visit to the store.
 
A store wants to categorize its members. There are three tiers:
- **"Diamond"**: if the conversion rate is greater than or equal to 80.
- **"Gold"**: if the conversion rate is greater than or equal to 50 and less than 80.
- **"Silver"**: if the conversion rate is less than 50.
- **"Bronze"**: if the member never visited the store.

The conversion rate of a member is (100 * total number of purchases for the member) / total number of visits for the member.

**Write a solution to report the id, the name, and the category of each member.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Members table:**

| member_id | name    |
|-----------|---------|
| 9         | Alice   |
| 11        | Bob     |
| 3         | Winston |
| 8         | Hercy   |
| 1         | Narihan |

**Visits table:**
| visit_id | member_id | visit_date |
|----------|-----------|------------|
| 22       | 11        | 2021-10-28 |
| 16       | 11        | 2021-01-12 |
| 18       | 9         | 2021-12-10 |
| 19       | 3         | 2021-10-19 |
| 12       | 11        | 2021-03-01 |
| 17       | 8         | 2021-05-07 |
| 21       | 9         | 2021-05-12 |

**Purchases table:**
| visit_id | charged_amount |
|----------|----------------|
| 12       | 2000           |
| 18       | 9000           |
| 17       | 7000           |

**Output:** 
| member_id | name    | category |
|-----------|---------|----------|
| 1         | Narihan | Bronze   |
| 3         | Winston | Silver   |
| 8         | Hercy   | Diamond  |
| 9         | Alice   | Gold     |
| 11        | Bob     | Silver   |

**Explanation:** 
- User Narihan with id = 1 did not make any visits to the store. She gets a Bronze category.
- User Winston with id = 3 visited the store one time and did not purchase anything. The conversion rate = (100 * 0) / 1 = 0. He gets a Silver category.
- User Hercy with id = 8 visited the store one time and purchased one time. The conversion rate = (100 * 1) / 1 = 1. He gets a Diamond category.
- User Alice with id = 9 visited the store two times and purchased one time. The conversion rate = (100 * 1) / 2 = 50. She gets a Gold category.
- User Bob with id = 11 visited the store three times and purchased one time. The conversion rate = (100 * 1) / 3 = 33.33. He gets a Silver category.

In [0]:
members_data_2051 = [
    (9, "Alice"),
    (11, "Bob"),
    (3, "Winston"),
    (8, "Hercy"),
    (1, "Narihan"),
]

members_columns_2051 = ["member_id", "name"]
members_df_2051 = spark.createDataFrame(members_data_2051, members_columns_2051)
members_df_2051.show()

visits_data_2051 = [
    (22, 11, "2021-10-28"),
    (16, 11, "2021-01-12"),
    (18, 9, "2021-12-10"),
    (19, 3, "2021-10-19"),
    (12, 11, "2021-03-01"),
    (17, 8, "2021-05-07"),
    (21, 9, "2021-05-12"),
]

visits_columns_2051 = ["visit_id", "member_id", "visit_date"]
visits_df_2051 = spark.createDataFrame(visits_data_2051, visits_columns_2051)
visits_df_2051.show()

purchases_data_2051 = [
    (12, 2000),
    (18, 9000),
    (17, 7000),
]

purchases_columns_2051 = ["visit_id", "charged_amount"]
purchases_df_2051 = spark.createDataFrame(purchases_data_2051, purchases_columns_2051)
purchases_df_2051.show()

+---------+-------+
|member_id|   name|
+---------+-------+
|        9|  Alice|
|       11|    Bob|
|        3|Winston|
|        8|  Hercy|
|        1|Narihan|
+---------+-------+

+--------+---------+----------+
|visit_id|member_id|visit_date|
+--------+---------+----------+
|      22|       11|2021-10-28|
|      16|       11|2021-01-12|
|      18|        9|2021-12-10|
|      19|        3|2021-10-19|
|      12|       11|2021-03-01|
|      17|        8|2021-05-07|
|      21|        9|2021-05-12|
+--------+---------+----------+

+--------+--------------+
|visit_id|charged_amount|
+--------+--------------+
|      12|          2000|
|      18|          9000|
|      17|          7000|
+--------+--------------+



In [0]:
visits_with_purchases_df_2051 = visits_df_2051\
                                    .join(purchases_df_2051, "visit_id", "left")

In [0]:
member_stats_df_2051 = visits_with_purchases_df_2051\
                            .groupBy("member_id") \
                                .agg(
                                    countDistinct("visit_id").alias("total_visits"),
                                    countDistinct(when(col("charged_amount").isNotNull(), 
                                                       col("visit_id"))).alias("total_purchases")
                                    )\
                                        .withColumn( "conversion_rate", (col("total_purchases") * 100 / col("total_visits")))

In [0]:
final_df_2051 = members_df_2051\
                    .join(member_stats_df_2051, "member_id", "left") \
                        .withColumn("total_visits", when(col("total_visits").isNull(), lit(0)).otherwise(col("total_visits"))) \
                            .withColumn("conversion_rate", when(col("conversion_rate").isNull(), lit(-1)).otherwise(col("conversion_rate")))

In [0]:
final_df_2051\
    .withColumn( "category",
                when(col("total_visits") == 0, "Bronze")
                .when(col("conversion_rate") >= 80, "Diamond")
                .when((col("conversion_rate") >= 50) & (col("conversion_rate") < 80), "Gold")
                .when(col("conversion_rate") < 50, "Silver")
                )\
                    .select("member_id", "name", "category").show()

+---------+-------+--------+
|member_id|   name|category|
+---------+-------+--------+
|        9|  Alice|    Gold|
|       11|    Bob|  Silver|
|        3|Winston|  Silver|
|        8|  Hercy| Diamond|
|        1|Narihan|  Bronze|
+---------+-------+--------+

